In [ ]:
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 1.6.0

In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var trainDataPath = "./data/iris-train.txt";
var testDataPath = "./data/iris-test.txt";

In [ ]:
public class IrisData
{
    [LoadColumn(0)]
    public float Label;

    [LoadColumn(1)]
    public float SepalLength;

    [LoadColumn(2)]
    public float SepalWidth;

    [LoadColumn(3)]
    public float PetalLength;

    [LoadColumn(4)]
    public float PetalWidth;
}

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
var trainingDataView = mlContext.Data.LoadFromTextFile<IrisData>(trainDataPath, hasHeader: true);
var testDataView = mlContext.Data.LoadFromTextFile<IrisData>(testDataPath, hasHeader: true);

In [ ]:
var dataProcessPipeline = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "KeyColumn", inputColumnName: nameof(IrisData.Label))
                         .Append(mlContext.Transforms.Concatenate("Features", nameof(IrisData.SepalLength),
                                                                              nameof(IrisData.SepalWidth),
                                                                              nameof(IrisData.PetalLength),
                                                                              nameof(IrisData.PetalWidth))
                                                                              .AppendCacheCheckpoint(mlContext));

In [ ]:
var trainer = mlContext.MulticlassClassification.Trainers.NaiveBayes(labelColumnName: "KeyColumn", featureColumnName: "Features")
              .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: nameof(IrisData.Label), inputColumnName: "KeyColumn"));

var trainingPipeline = dataProcessPipeline.Append(trainer);

In [ ]:
Console.WriteLine("=============== Training the model ===============");
var trainedModel = trainingPipeline.Fit(trainingDataView);

=============== Training the model ===============


In [ ]:
Console.WriteLine("===== Evaluating Model's accuracy with Test data =====");
var predictions = trainedModel.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions);

Console.WriteLine($"MacroAccuracy: {metrics.MacroAccuracy:P2}");
Console.WriteLine($"MicroAccuracy: {metrics.MicroAccuracy:P2}");

===== Evaluating Model's accuracy with Test data =====
MacroAccuracy: 33,33%
MicroAccuracy: 34,48%
